Notebook to create embeddings of images from a parquet file, created with the get_images notebook, containing images for recipes.

In [ ]:
import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from PIL import Image, UnidentifiedImageError
import os
from scipy import sparse
from torchvision.datasets import DatasetFolder, VisionDataset
from torchvision import models as tv_models
import torchvision.transforms.v2 as T
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
bs=2**5

In [ ]:
def write_ims(root=''):
    os.mkdir(root+'images')
    os.mkdir(root+'images/0')
    res = {'exceptions':[], 'images':[]}
    im_df = pl.scan_parquet(path+'images_df.parquet').select(pl.exclude('AggregatedRating'))
    df_len = im_df.select(pl.count()).collect()['count'][0]
    for recipe, image in tqdm(im_df.collect().iter_rows(), total=df_len, miniters=1000):
        path = f'{root}images/0/{recipe}.jpg'
        if image is not None:
            with open(path, 'wb') as f:
                f.write(image)
            try:
                Image.open(path)
            except UnidentifiedImageError:
                res['exceptions'].append(recipe)
                os.remove(path)
            else:
                res['images'].append(recipe)

In [ ]:
def to_parquet(embs, recipes, name):
    pd.DataFrame(
        np.vstack(embs),
        index = np.hstack(recipes),
        columns=[f'{name}_{i}' for i in range(embs[0].shape[1])]
    ).to_parquet(
        f'{name}.parquet'
    )

In [ ]:
def loader(path):
    return Image.open(path), float(path.split('/')[-1][:-4])
tr = T.Compose([T.ToImageTensor(),T.CenterCrop([256]),T.ToDtype(torch.float)])

In [ ]:
def get_embs(name):
    try:
        from sentence_transformers import SentenceTransformer
    except ModuleNotFoundError:
        !pip install sentence-transformers
        from sentence_transformers import SentenceTransformer
    model = SentenceTransformer(name).to(device)
    recipes = []
    embs = []
    transform = T.Compose([T.ToImageTensor(), T.CenterCrop([256])])
    ds = DatasetFolder('images', loader, 'jpg', transform)
    dl=DataLoader(ds, batch_size=bs, num_workers=4)
    to_im = T.ToPILImage()
    with torch.no_grad():
        for (ims, recipe), _ in tqdm(dl):
            ims = [to_im(im) for im in ims]
            embs.append(model.encode(ims, device=device, show_progress_bar=False))
            recipes.append(recipe.numpy())
    to_parquet(embs, recipes, name)

In [ ]:
write_ims()

In [ ]:
get_embs("clip-ViT-B-32")

In [ ]:
[os.remove('images/0/'+f) for f in os.listdir('images/0')]
os.removedirs('images/0')